#### Word2Vec
- 단어 간의 유사성 측정하기 위해 분포 가설을 기반 개발
    - 분포 가설: 같은 문맥에서 함께 자주 나타나는 단어는 서로 유사한 의미를 가질 가능성이 높다는 가정
        - 동시 발생 확률 분포를 이용해 단어 간의 유사성 측정
- 유사한 문맥에서 등장하는 단어는 비슷한 벡터 공강산 위치를 갖게 됨

#### 단어 벡터화
- 방법
    - 희소 표현: 데이터를 고차원의 벡터로 표현 벡터 대부분 요소가 0으로 채워짐 
        - ex) 어휘 사전의 크기가 10,000이고, 특정 문서에 100개의 고유 단어만 포함되어 있다면, 이 문서는 100개의 비-제로 값과 9,900개의 제로 값을 가진 10,000차원 벡터로 표현
        - BoW: 순서에 관계없이 단어의 출현 빈도로만 표현하는 것
        - 원-핫 인코딩: 각 단어를 고유한 색인 값으로 매핑
        - TF-IDF: 단어의 빈도수와 함께 단어가 전체 문서 집합에서 얼마나 고유한지를 함께 고려하는 방법
        - 장점: 명확한 관계를 제공
        - 단점: 고차원이며, 대부분의 값이 0으로 채워져 있어 메모리와 계산 효율이 낮고 단어의 의미적 관계를 포착하지 못하는 경우가 많음
    - 밀집 표현
        - Word2Vec
        - 장점: 의미적 유사성을 기반으로 한 단어 간의 관계를 잘 포착, 계산 효율이 높음
        - 단점: 특징 차원이 구체적으로 어떤 의미를 가지는지 해석하기 어려울 수 있음

    

#### CBoW
- 주변에 단어를 가지고 중간에 있단 단어 예측하는 방법
- 중심 단어(Center Word): 예측할 단어
- 주변 단어(Context Word): 예측에 사용되는 단어
- sliding window를 이용해 이동해 가며 학습 -> 한 번의 학습으로 여러 개의 중심 단어, 그에 대한 주변 단어 학습


#### Skip-gram
- 중심 단어를 입력으로 주변 단어를 예측하는 모델
- 중심 단어와 주변 단어를 하나의 쌍으로 하여 여러 학습 데이터가 만들어짐
- 말뭉치가 커지면 단어 사전 커져 Word2Vec 모델 학습 시 학습 속도 느려지는 문제
    - 해결방법
        - 계층적 소프트맥스
            - 출력층 이진 트리 구조 표현
            - 자주 등장하는 단어 상위 노드, 드물게 등장하는 단어 하위 노드
        - 네거티브 샘플링
            - 전체 단어 집합에서 일부 단어를 샘플링하여 오답 단어로 사용
            - 학습 윈도 내에 등장하지 않는 단어 추출 후 정답 단어와 소프트맥스 연산 수행 (전체 단어 확률 계산할 필요가 없어짐)

In [ ]:
# 임베딩 클래스
import torch

embedding = torch.nn.Embedding(
    num_embeddings, # 임베딩 수: 이산 변수의 개수로 단어 사전의 크기
    embedding_dim,  # 임베딩 차원: 임베딩 벡터의 크기
    padding_idx=None, # 패딩 인덱스: 패딩 토큰의 인덱스를 지정 / 병렬 처리를 위해 문장 길이 동일해야하므로 길이를 맞추는 역할 / 임베딩 수보다 작아야 함
    max_norm=None, # 노름 타입: 임베딩 벡터의 크기 제한 default: L2 정규화 
    norm_type=2.0 # 최대 노름: 임베딩 벡터의 최대 크기 (각 임베딩 벡터의 크기가 최대 노름 값 이상이면 임베딩 벡터를 최대 노름 크기로 잘라냄)
)

In [3]:
# 기본 skip-gram 클래스

from torch import nn

class VanillaSkipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim
        )
        self.linear = nn.Linear(
            in_features=embedding_dim,
            out_features=vocab_size
        )

    def forward(self, input_ids):
        embedding = self.embedding(input_ids)
        output = self.linear(embedding)
        return output

In [4]:
# 영화 리뷰 데이터세트 전처리

import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt

corpus = Korpora.load('nsmc')
corpus = pd.DataFrame(corpus.test)

tokenizer = Okt()
tokens = [tokenizer.morphs(review) for review in corpus.text]
print(tokens[:3])


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/joyoungjun/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/jo

In [6]:
# 단어 사전 구축

from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens): # n_vocab: 구축할 단어 사전의 크기 초과 시 가장 많이 등장한 토큰 순서로 사전 구축, special_tokens: 특별한 의미를 갖는 토큰 <unk>:oov대응 토큰
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens) # 업데이트 진행
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

vocab = build_vocab(corpus=tokens, n_vocab=5000, special_tokens=["<unk>"]) # n_vocab: 사전 크기
token_to_id = {token: idx for idx, token in enumerate(vocab)}

id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<unk>', '.', '이', '영화', '의', '..', '가', '에', '...', '을']
5001


In [ ]:
# Skip-gram의 단어 쌍 추출

# ㄹㄹㄹ

# skip-gram 모델의 입력 데이터로 사용할 수 있게 전처리함
def get_word_pairs(tokens, window_size):
    pairs = []
    for sentence in tokens:
        sentence_length = len(sentence)
        for idx, center_word in enumerate(sentence):
            window_start = max(0, idx - window_size)
            window_end = min(sentence_length, idx + window_size + 1)
            center_word = sentence[idx]
            context_words = sentence[window_start:idx] + sentence[idx+1:window_end]
            for context_word in context_words:
                pairs.append([center_word, context_word])
    return pairs

word_pairs = get_word_pairs(tokens, window_size=2)
print(word_pairs[:5])